<a href="https://colab.research.google.com/github/MarioRMReis/Sentinel_Images/blob/main/EE_ImageRequest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run colab locally.

 - pip install jupyter_http_over_ws  
 - jupyter serverextension enable --py jupyter_http_over_ws
 - jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8888 --NotebookApp.port_retries=0

# Import the required libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
import geemap
import os
import cv2
import requests
import math
from pykml import parser
%matplotlib inline

# Log-in into Earth Engine


> Clic the link and get the verification code, paste it and hit enter



In [2]:
import ee
# Trigger the authentication flow.
ee.Authenticate()
 
# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AVHEtk5xJ9Rlafid4ERGpGkHhjfmUWhfJkh-g6Zu8n6htU9fQgtJplHk4mw

Successfully saved authorization token.


# Parameters for the image request (C: drive path, Starting data, Ending date, Max number of images)

In [3]:
C_drivePath = 'C:/Users/mario/Documents/Mario/'
folder_dataset = C_drivePath + 'Sentinel/dataset'
starting_date = '2017-01-01'
ending_date = '2023-12-31'
nimgs = 2190

# Read KML files and extract all regions of interest 

In [4]:
aois = []
aoi_names = []
kml_files = os.listdir(C_drivePath + 'Sentinel/roi/')

for kml in kml_files:
  with open(C_drivePath + 'Sentinel/roi/' + kml, 'r') as f:
    root = parser.parse(f).getroot()
  namespace = {"kml": 'http://www.opengis.net/kml/2.2'}
  pms = root.xpath(".//kml:Placemark[.//kml:Polygon]", namespaces=namespace)
  roi_string = []
  for p in pms:
    if hasattr(p, 'MultiGeometry'):
      for poly in p.MultiGeometry.Polygon:
        roi_string.append(poly.outerBoundaryIs.LinearRing.coordinates)
    else:
      roi_string.append(p.Polygon.outerBoundaryIs.LinearRing.coordinates)

  dot = kml_files[0].find('.')
  name_kml = kml_files[0][:dot] + '-'
  for jdx, r in enumerate(roi_string):
    aoi_names.append(name_kml + str(jdx))
    roi_str = str(r).split(' ')
    aux = []
    for idx, rs in enumerate(roi_str):
      if idx == 0:
        n = rs[7::].split(',')
        aux.append([float(n[i]) for i in range(len(n)-1)])
      elif idx == (len(roi_str)-1): 
        print(rs)
      else:
        n = rs.split(',')
        aux.append([float(n[i]) for i in range(len(n)-1)])
    aois.append([aux])

# Get the fitted Area Of Interest for a 128x128 
> The aoi will be at the center of the 128x128 image



In [5]:
def get_squares(aois):
    # 0.011377373345324189 0.014894925631009616 -> Height, Width
    S128 = [0.014854925631009616, 0.011377373345324189]
    new_aois = []
    h_aux = S128[1]/2
    w_aux = S128[0]/2

    for aoi in aois:
      # Find centeroid ----------------------------
      siz = len(aoi[0])
      list_width = []
      list_height = []
      for i in range(siz): 
        list_width.append(aoi[0][i][0])
        list_height.append(aoi[0][i][1])

      max_width = max(list_width)
      min_width = min(list_width)
      max_height = max(list_height)
      min_height = min(list_height)
      
      centroid_width = (max_width+min_width)/2
      centroid_height = (max_height+min_height)/2

      aoi_new = [[[(centroid_width-w_aux), (centroid_height+h_aux)], [(centroid_width-w_aux), (centroid_height-h_aux)], [(centroid_width+w_aux), (centroid_height-h_aux)], [(centroid_width+w_aux), (centroid_height+h_aux)]]]
      #inside, split_aois = check_if_inside(aoi_new, aoi)
      #if inside:
      new_aois.append(aoi_new)
      size = [128,128]
      #else:
        #for saois in split_aois:
          #new_aois.append(saois)

      
      #ans = image_ratio(S128, [list_width, list_height])
    return new_aois, size

# For all AOIs Create the mask and save it in the mask folder for the aoi

In [6]:
def get_mask(path, aoi, size):
  # Inputs:
  #     - path -> path to the aoi saved images
  #     - aoi -> Area of interest of the landfield
  #     - size = [sizeX, sizeY] -> size of the mask, (ie: 128x128, 256x256, 354x128...)
  aoi_mask = ee.Geometry.Polygon(aoi,None,False)
  ffa_s2 = ee.ImageCollection('COPERNICUS/S2') \
                        .filterBounds(aoi_mask) \
                        .filterDate(ee.Date(starting_date), ee.Date(ending_date))
  colList = ffa_s2.toList(30)
  # This part get's the land area image needed to create the mask --------------
  img = ee.Image(colList.get(17)).double().clip(aoi_mask)
  rgb = ['B4','B3','B2']
  url = img.getThumbURL({"min":-200000, "max":-200000,"bands":rgb})

  img_data = requests.get(url).content

  img_aux = cv2.imdecode(np.frombuffer(img_data, np.uint8), -1)

  # Create mask ----------------------------------------------------------------
  # img_aux.shape -> [x,y,z]
  x = img_aux.shape[0]
  y = img_aux.shape[1]
  x_sum = math.ceil((size[0]-x)/2)
  y_sum = math.ceil((size[1]-y)/2)

  img_zeros = np.zeros([size[0], size[1], 3])
  img_zeros[x_sum:(x_sum+x), y_sum:(y_sum+y), :] = img_aux[:,:,0:3]
  try:
    os.makedirs(path)
    cv2.imwrite(path +'mask.jpg', img_zeros)
  except:
    cv2.imwrite(path +'mask.jpg', img_zeros)

In [7]:
aoi_square, size =  get_squares(aois)

for idx, a in enumerate(aois):
  path = folder_dataset + '/' + aoi_names[idx] + '/Mask/'
  get_mask(path, a, size)

# Extract Images from Sentinel-1

In [8]:
def ExportCol_Sentinel1(roi, channel, min, max, idx, jdx):
  try:
    for i in range(0,nimgs,1):
      aoi_geometry = ee.Geometry.Polygon(roi ,None,False)
      ffa_s = ee.ImageCollection('COPERNICUS/S1_GRD') \
                        .filterBounds(aoi_geometry) \
                        .filterDate(ee.Date(starting_date), ee.Date(ending_date))
                        
      colList = ffa_s.toList(nimgs); 

      img = ee.Image(colList.get(i)).double().clip(aoi_geometry)
      # Exeption beacause to get a RGB channel with the Sentinel-1 we need to correspond each channel and compute the last
      if channel == 'RGB':
        imgR = img.select('VV')
        imgG = img.select('VH')
        imgB = img.select('VV').divide(img.select('VH'))
        url = ee.Image.rgb(imgR,imgG,imgB).getThumbURL({'min': [min[0], min[1], 0], 'max': [0, 0, 2]})
      else:
        url = img.select(channel).getThumbURL({'min': min[jdx], 'max': max[jdx]})

      id = img.id().getInfo()

      img_data = requests.get(url).content

      path = folder_dataset + '/' + aoi_names[idx] + '/' + id + '/'
      # Creates the folder but the flag need to be turned off beacause it will try to makedir after the creation
      try:
        os.makedirs(path)
        with open(path  + channel + '.tiff', 'wb') as handler:
          handler.write(img_data)
          #create_folders = False
      except:
        # This just saves the image in the correct folder with the name 'id' (var->'id')
        with open(path + channel + '.tiff', 'wb') as handler:
          handler.write(img_data)
  except:
    return



> The request of the images from Sentinel-1 all aoi's 



In [ ]:
aoi_bands = ee.Geometry.Polygon(aoi[0],None,False)

ffa_db = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD') 
                       .filterBounds(aoi_bands)
                       .first() 
                       .clip(aoi_bands))

# Variables needed to save images
bands_s1 = ffa_db.bandNames().getInfo()
bands_s1.remove('angle')
bands_s1.append('RGB')

# Defined values to be the best for these bands
min = [-14, -25]
max = [-7] * 2
#-------------------
for idx, a in enumerate(aoi_square):
  for jdx, b in enumerate(bands_s1):

    ExportCol_Sentinel1(a, b, min, max, idx, jdx)

# Extract images from Sentinel-2



> Function that evaluetes the percentage of image incomplete by counting the amout of pixels that are zero, it's perfectly ok to have some images that have pixels equal to zero to acount for this we are going to give it a margin like 99% complete even that 1% it's not alot of pixels. Might be lower depending on the results.


In [9]:
def Check_image_notComplete(idx, image, percentage):
  # B2 - using this band
  decoded = cv2.imdecode(np.frombuffer(image, np.uint8), -1)
  nonZero_percentage = ((np.count_nonzero(decoded)*100)/decoded.size)

  if nonZero_percentage < percentage:
    return True
  else:
    return False

In [10]:
def ExportCol_Sentinel2(roi, channel,  min, max, idx, jdx, percentage, incomplete_images):
  for i in range(0,nimgs,1):
    try:
      aoi_geometry = ee.Geometry.Polygon(roi ,None,False)
      ffa_s = ee.ImageCollection('COPERNICUS/S2') \
                        .filterBounds(aoi_geometry) \
                        .filterDate(ee.Date(starting_date), ee.Date(ending_date)) \
                        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
      colList = ffa_s.toList(ffa_s.size());                   

      img = ee.Image(colList.get(i)).double().clip(aoi_geometry)
      # Exeption beacause to get a RGB channel with the Sentinel-1 we need to correspond each channel and compute the last
      if channel == 'RGB':
        rgb = ['B4','B3','B2']
        url = img.getThumbURL({"min":min[jdx], "max":max[jdx],"bands":rgb})
      else:
        url = img.getThumbURL({"min":min[jdx], "max":max[jdx],"bands":channel})

      id = img.id().getInfo()
      id_short = id[16::]

      img_data = requests.get(url).content
      
      # Check the image
      if channel == 'B2':
        margin = Check_image_notComplete(i, img_data, percentage)
        if margin == True:
          incomplete_images.append(i)
      
      # Image path
      path = folder_dataset + '/' + aoi_names[idx] + '/Sentinel-2/' + id_short + '/'

      if i not in incomplete_images:
        # Creates the folder but the flag need to be turned off beacause it will try to makedir after the creation
        try:
          os.makedirs(path)
          with open(path + channel + '.tiff', 'wb') as handler:
            handler.write(img_data)
        except:
          # This just saves the image in the correct folder with the name 'id' (var->'id')
          with open(path + channel + '.tiff', 'wb') as handler:
            handler.write(img_data)
      else:
        path = folder_dataset + '/review/' + aoi_names[idx] + '/' + id_short + '/'
        try:
          os.makedirs(path)
          with open(path + channel + '.tiff', 'wb') as handler:
            handler.write(img_data)
        except:
          # This just saves the image in the correct folder with the name 'id' (var->'id')
          with open(path + channel + '.tiff', 'wb') as handler:
            handler.write(img_data)
    except:
      if channel == 'B2':
        return incomplete_images
      else:
        return



> The request, cloud coverage < 20% 



In [11]:
aoi_bands = ee.Geometry.Polygon(aois[0],None,False)

ffa_s2 = ee.Image(ee.ImageCollection('COPERNICUS/S2') 
                       .filterBounds(aoi_bands) 
                       .first() 
                       .clip(aoi_bands))

bands_s2 = ffa_s2.bandNames().getInfo()
aux_bands_s2 =  bands_s2.copy();
for x in aux_bands_s2: 
  if x in  ['QA10','QA20','QA60','B10']: 
    bands_s2.remove(x) #bands_s2.remove(['QA10','QA20','QA60','B10'])
bands_s2.append('RGB')

# Defined best values for these bands
min = [0] * len(bands_s2)
max = [2700, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 2000, 3200, 2500, 3000]

bands_s2.insert(0,bands_s2.pop(bands_s2.index('B2')))

#max = [3000]
#bands_s2 = ['RGB']
#aoi_square = [aoi_square[1]]
#aoi_names = ['green-botics-fields_2-1']
#incomplete_images_B2 = np.load((folder_dataset + '/green-botics-fields_2-1/inc_images.npy'))

for idx, a in enumerate(aoi_square):
  for jdx, b in enumerate(bands_s2):
    if b == 'B2':
      incomplete_images = []
      incomplete_images_B2 =ExportCol_Sentinel2(a, b, min, max, idx, jdx, 98, incomplete_images)
    else:
      ExportCol_Sentinel2(a, b, min, max, idx, jdx, 98, incomplete_images_B2)